# Train Model
The train_model.ipynp notebook a neural network to recognize ten different hand gestures.  
The trained model will be stored in models/model_<\hash>.h5. The training results will be stored in models/results/results_<\hash>.md.  
The notebook assumes that the training and test data are available in data/train/training_data.h5 and data/test/test_data.h5.

#### (Optional) Google Colab setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Install requirements
!pip install -r /content/drive/MyDrive/GestureDetection_ML_ITA/requirements.txt
# Go to src folder
%cd /content/drive/MyDrive/GestureDetection_ML_ITA/src

#### Import the necessary libraries

In [ ]:
import model
import tensorflow as tf

#### (Optional) Confirm GPU connection

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#### Preparations for the training process

In [ ]:
# Define the paths to the training and test HDF5 files
training_h5_path = '../data/train/training_data.h5'
test_h5_path = '../data/test/test_data.h5'
# Define the hyperparameter search space
batch_size = [32, 64, 128, 256, 512, 1024]
epochs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
hidden_layers = [1, 2, 3]
neurons_layer1 = [16, 32, 64, 128, 256]
neurons_layer2 = [16, 32, 64, 128, 256]
neurons_layer3 = [16, 32, 64, 128, 256]
batch_normalization = [True, False]
dropout = [True, False]
param_distributions = dict(
    batch_size=batch_size,
    epochs=epochs,
    model__hidden_layers=hidden_layers,
    model__neurons_layer1=neurons_layer1,
    model__neurons_layer2=neurons_layer2,
    model__neurons_layer3=neurons_layer3,
    model__batch_normalization=batch_normalization,
    model__dropout=dropout
)
# Prepare the data
x_train, y_train, x_test, y_test = model.prepare_data(training_h5_path, test_h5_path)

#### Train and evaluate the model with bayesian optimization

In [ ]:
model.train_and_evaluate_model(x_train, y_train, x_test, y_test, param_distributions, n_iter=20, cv=4, n_jobs=1, verbose=1)

#### Train and evaluate the final model based on the best hyperparameters

In [ ]:
# Define the best hyperparameters dict
best_distributions = dict(
    batch_size=256,
    epochs=80,
    hidden_layers=2,
    neurons_layer1=64,
    neurons_layer2=128,
    neurons_layer3=64,
    batch_normalization=True,
    dropout=True
)
# Retrain the model with the best hyperparameters  
model.train_and_evaluate_custom_model(x_train, y_train, x_test, y_test, best_distributions)